In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pickle
df = pd.read_csv("shuffled-full-set-hashed.csv", header=None)


In [2]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
encoder = pickle.load(open('../server/webservice/pickles/encoder.pkl', 'rb'))
vectorizer = pickle.load(open('../server/webservice/pickles/vectorizer.pkl', 'rb'))
lsa = pickle.load(open('../server/webservice/pickles/lsa.pkl', 'rb'))
knn_lsa = pickle.load(open('../server/webservice/pickles/knn_lsa.pkl', 'rb'))

In [29]:
smaller_vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000,
                             min_df=10, use_idf=True)

In [32]:
df_nona = df.dropna()
smaller_vectorizer.fit(df_nona[1])


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.5, max_features=10000,
                min_df=10, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [33]:
pickle.dump(smaller_vectorizer, open('smaller_vectorizer.pkl', 'wb'))

In [37]:
from sklearn import model_selection
train_x, test_x, train_y, test_y = model_selection.train_test_split(df_nona[1], df_nona[0],test_size=0.2)
train_x_tfidf = smaller_vectorizer.transform(train_x)

In [38]:
svd = TruncatedSVD(100)
lsa = make_pipeline(svd, Normalizer(copy=False))

# Run SVD on the training data, then project the training data.
train_x_lsa = lsa.fit_transform(train_x_tfidf)

In [39]:
pickle.dump(lsa, open('new_lsa.pkl', 'wb'))
